# Artificial Neural Network

### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from pathlib import Path

### Importing the dataset

In [2]:
current_dir = Path.cwd()
parent_dir = current_dir.parent

In [3]:
#Access input folder
input_dir1 = Path (parent_dir/"Pressure_profiles_sectioned_1")
print ("input",input_dir1)

#Access folders inside input folder
input_dir2 =  [folder for index,folder in enumerate (input_dir1.iterdir()) if folder.is_dir() and index % 2 == 0]
print ("2",input_dir2)

input H:\Chamodi\Machine Learning\Pressure_history\Pressure_profiles_sectioned_1
2 [WindowsPath('H:/Chamodi/Machine Learning/Pressure_history/Pressure_profiles_sectioned_1/L13_1'), WindowsPath('H:/Chamodi/Machine Learning/Pressure_history/Pressure_profiles_sectioned_1/L13_13'), WindowsPath('H:/Chamodi/Machine Learning/Pressure_history/Pressure_profiles_sectioned_1/L13_17'), WindowsPath('H:/Chamodi/Machine Learning/Pressure_history/Pressure_profiles_sectioned_1/L13_21'), WindowsPath('H:/Chamodi/Machine Learning/Pressure_history/Pressure_profiles_sectioned_1/L13_25'), WindowsPath('H:/Chamodi/Machine Learning/Pressure_history/Pressure_profiles_sectioned_1/L13_5'), WindowsPath('H:/Chamodi/Machine Learning/Pressure_history/Pressure_profiles_sectioned_1/L13_9')]


In [4]:
# Access each folder inside input folder
dataset = pd.DataFrame(columns = ['Charge_mass', 'Standoff_distance', 'Time', 'Pressure'])
df_list = []
for folder in input_dir2:
    
    # Make a list of data file names
    files = list(folder.rglob("*.xlsx"))
    
    for file in files:
        df_list.append(pd.read_excel(file))

dataset = pd.concat(df_list, ignore_index = True)

In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303729 entries, 0 to 303728
Data columns (total 4 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Charge_mass        303729 non-null  float64
 1   Standoff_distance  303729 non-null  int64  
 2   Time               303729 non-null  float64
 3   Pressure           303729 non-null  float64
dtypes: float64(3), int64(1)
memory usage: 9.3 MB


In [6]:
y = dataset['Pressure']
X = dataset.drop(['Pressure'], axis=1)
print(X.shape, y.shape)

(303729, 3) (303729,)


In [7]:
# convert to numpy array
X = np.array(X)
y = np.array(y)

### Splitting the dataset into the Training set and Test set

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [13]:
print(X_train.shape)

(212610, 3)


### Hyperparameter tuning 1 - layers, neurons, activation function

In [14]:
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor as KR
import math
def FindLayerNodesLinear(n_layers, first_layer_nodes, last_layer_nodes):
    layers = []
    
    nodes_increment = (last_layer_nodes - first_layer_nodes)/ (n_layers-1)
    nodes = first_layer_nodes
    for i in range(1, n_layers+1):
        layers.append(math.ceil(nodes))
        nodes = nodes + nodes_increment
    
    return layers

In [15]:
from tensorflow.keras.optimizers import Adam
def create_model(n_layers, first_layer_nodes, last_layer_nodes, activation_func):
    model = Sequential()
    n_nodes = FindLayerNodesLinear(n_layers, first_layer_nodes, last_layer_nodes)
    for i in range(1, n_layers):
        if i==1:
            model.add(Dense(units = first_layer_nodes,  input_shape=(X_train.shape[1],), activation=activation_func))
        else:
            model.add(Dense(n_nodes[i-1], activation=activation_func))
            
    #Finally, the output layer should have a single node in binary classification
    model.add(Dense(1, activation='linear'))
    opt = Adam(learning_rate=0.001)
    model.compile(optimizer = opt, loss = 'mean_squared_error', metrics = ['mae'])
    return model

##Wrap model into scikit-learn
model = KR(build_fn=create_model, epochs = 500, batch_size = 50)

C:\Users\z5379606\AppData\Local\Temp\ipykernel_25968\2168085556.py:18: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KR(build_fn=create_model, epochs = 500, batch_size = 50)


In [17]:
activation_funcs = ['relu', 'softplus'] 
param_grid = dict(n_layers=[3],
                  first_layer_nodes = [300],
                  last_layer_nodes = [200],
                  activation_func = activation_funcs)
grid = GridSearchCV(estimator = model, param_grid = param_grid,n_jobs=-1, cv=2, scoring = 'r2')
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/500
4253/4253 [==============================] - 11s 2ms/step - loss: 2589999104.0000 - mae: 27243.1250
Epoch 2/500
4253/4253 [==============================] - 10s 2ms/step - loss: 1344568320.0000 - mae: 5212.7090
Epoch 3/500
4253/4253 [==============================] - 11s 2ms/step - loss: 1337531776.0000 - mae: 4791.1865
Epoch 4/500
4253/4253 [==============================] - 10s 2ms/step - loss: 1336485248.0000 - mae: 4839.5942
Epoch 5/500
4253/4253 [==============================] - 10s 2ms/step - loss: 1335401984.0000 - mae: 4881.2749
Epoch 6/500
4253/4253 [==============================] - 10s 2ms/step - loss: 1334883584.0000 - mae: 4932.6416
Epoch 7/500
4253/4253 [==============================] - 10s 2ms/step - loss: 1334776576.0000 - mae: 5015.9780
Epoch 8/500
4253/4253 [==============================] - 10s 2ms/step - loss: 1334863488.0000 - mae: 4991.4126
Epoch 9/500
4253/4253 [==============================] - 10s 2ms/step - loss: 1334811776.0000 - mae: 5057.6743


### Hyperparameter tuning - batch size, epoch, optimizer, learning rate

In [9]:
# Function to create model, required for KerasRegressor
def create_model(optimizer):
 # create model
 model = Sequential()
 model.add(Dense(units=65, input_shape=(X_train.shape[1],), activation='relu'))
 model.add(Dense(units=48, activation='relu'))
 model.add(Dense(units=30, activation='relu'))
 model.add(Dense(units=1, activation='linear'))
 # Compile model
 model.compile(optimizer = optimizer, loss = 'mean_squared_error', metrics = ['mae'])
 return model

In [ ]:
from scikeras.wrappers import KerasRegressor
# create model
model = KerasRegressor(model=create_model)

# define the grid search parameters
batch_size = [30,40,50]
epochs = [300, 400, 500]
optimizer = ['RMSprop', 'Adam', 'Adamax', 'Nadam']
learn_rate = [0.001, 0.01, 0.1]

# gridsearch
param_grid = dict(batch_size=batch_size, epochs=epochs, model__optimizer=optimizer, optimizer__learning_rate=learn_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=5, scoring = 'r2')
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/500
9/9 [==============================] - 1s 2ms/step - loss: 4.5365 - mae: 1.5545
Epoch 2/500
9/9 [==============================] - 0s 2ms/step - loss: 0.9797 - mae: 0.8338
Epoch 3/500
9/9 [==============================] - 0s 3ms/step - loss: 0.2384 - mae: 0.4261
Epoch 4/500
9/9 [==============================] - 0s 2ms/step - loss: 0.0723 - mae: 0.2175
Epoch 5/500
9/9 [==============================] - 0s 2ms/step - loss: 0.0195 - mae: 0.1090
Epoch 6/500
9/9 [==============================] - 0s 2ms/step - loss: 0.0147 - mae: 0.0876
Epoch 7/500
9/9 [==============================] - 0s 2ms/step - loss: 0.0068 - mae: 0.0611
Epoch 8/500
9/9 [==============================] - 0s 2ms/step - loss: 0.0033 - mae: 0.0426
Epoch 9/500
9/9 [==============================] - 0s 2ms/step - loss: 0.0015 - mae: 0.0333
Epoch 10/500
9/9 [==============================] - 0s 2ms/step - loss: 0.0010 - mae: 0.0259
Epoch 11/500
9/9 [==============================] - 0s 2ms/step - loss: 8.7465e